In [ ]:
#Z-Score Normalization
import scipy.io
import numpy as np
import matplotlib.pyplot as plt

# Load .mat file
mat_data = scipy.io.loadmat(r"C:\Users\HP\Downloads\H_017_000557.mat")

# Extract the signal (replace 'your_signal_variable' with the actual variable name)
signal = mat_data['B27B140816224010S'].flatten()

# Z-score normalization
mean_value = np.mean(signal)
std_dev = np.std(signal)

normalized_signal = (signal - mean_value) / std_dev

# Plot the original and normalized signals
plt.figure(figsize=(10, 6))

plt.subplot(2, 1, 1)
plt.plot(signal)
plt.title('Original Signal')
plt.xlabel('Sample')
plt.ylabel('Amplitude')

plt.subplot(2, 1, 2)
plt.plot(normalized_signal)
plt.title('Z-Score Normalized Signal')
plt.xlabel('Sample')
plt.ylabel('Amplitude')

plt.tight_layout()
plt.show()

#Save normalized file
mat_data['Normal'] = normalized_signal
scipy.io.savemat(r"C:\Users\HP\OneDrive\Desktop\normalized\Hpt.mat", mat_data)

In [ ]:
#Segmentation
import os
import scipy.io as sio

# Load .mat file
mat_file_path = r"C:\Users\HP\OneDrive\Desktop\normalized\Hpt.mat"
data = sio.loadmat(mat_file_path)

# Extract BCG signal
bcg_signal = data['Normal'].flatten()  # Assuming signal is stored in 'Normal'

# Set window size (assuming sampling rate is 100 Hz)
window_size = 30 * 100

# Specify the folder to save segments
save_folder = r"C:\Users\HP\OneDrive\Desktop\SEGMENTATIONS"

# Create the save folder if it doesn't exist
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# Loop through the signal to create segments
start_index = 0
segment_num = 1
while start_index + window_size <= len(bcg_signal):
    # Extract current segment
    segment = bcg_signal[start_index:start_index + window_size]

    # Create filename for the segment
    filename = os.path.join(save_folder, 'segment_Hpt{}.mat'.format(segment_num))

    # Save the segment in a separate .mat file
    sio.savemat(filename, {'segment': segment})

    # Update indices for next segment
    start_index += window_size
    segment_num += 1

# Display message
print('BCG signal segmentation complete. Segments saved in: ', save_folder)


In [ ]:
import os
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import pairwise_distances
from pyts.image import GramianAngularField, MarkovTransitionField

# Custom recurrence plot function
def recurrence_plot(s, eps=None, steps=None):
    if eps is None:
        eps = 0.1
    if steps is None:
        steps = 10

    # Calculate pairwise distances
    d = pairwise_distances(s[:, None])

    # Normalize distances by epsilon and threshold by steps
    d = d / eps
    d[d > steps] = steps

    # Normalize and shift to range [-1, 1]
    return d / 5. - 1

# Folder containing segmented BCG data files
folder_path = 'path/to/segmented_files/'

# Output folder to save the plot images
output_folder = 'path/to/plot_images/'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.mat'):
        # Load segmented BCG data
        file_path = os.path.join(folder_path, file_name)
        data = sio.loadmat(file_path)
        segment = data['segment'].flatten()

        # Convert data into GAF and MTF images
        gaf = GramianAngularField(method='summation')
        gaf_image = gaf.fit_transform(segment.reshape(1, -1))[0]

        mtf = MarkovTransitionField()
        mtf_image = mtf.fit_transform(segment.reshape(1, -1))[0]

        # Compute custom recurrence plot
        rp_image = recurrence_plot(segment)

        # Plot and save the individual images
        plt.figure(figsize=(8, 8))
        plt.imshow(gaf_image, cmap='viridis', origin='lower', aspect='auto')  # GAF colormap
        plt.title('GAF')
        plt.colorbar()
        plt.savefig(os.path.join(output_folder, 'GAF', f'{file_name[:-4]}.png'))
        plt.close()

        plt.figure(figsize=(8, 8))
        plt.imshow(rp_image, cmap='viridis', origin='lower', aspect='auto')  # RP colormap
        plt.title('RP')
        plt.colorbar()
        plt.savefig(os.path.join(output_folder, 'RP', f'{file_name[:-4]}.png'))
        plt.close()

        plt.figure(figsize=(8, 8))
        plt.imshow(mtf_image, cmap='viridis', origin='lower', aspect='auto')  # MTF colormap
        plt.title('MTF')
        plt.colorbar()
        plt.savefig(os.path.join(output_folder, 'MTF', f'{file_name[:-4]}.png'))
        plt.close()

        # Create multimodal image by averaging the three images
        multimodal_image = (gaf_image + rp_image + mtf_image) / 3

        # Plot and save the multimodal image
        plt.figure(figsize=(8, 8))
        plt.imshow(multimodal_image, cmap='viridis', origin='lower', aspect='auto')  # Multimodal image colormap
        plt.title('Multimodal Image')
        plt.colorbar()
        plt.savefig(os.path.join(output_folder, 'Multimodal', f'{file_name[:-4]}.png'))
        plt.close()